# Titanic dataset

In [2]:
import pandas as pd
train_data = pd.read_csv('train.csv')
test_X = pd.read_csv('test.csv')
test_y = pd.read_csv('gender_submission.csv')

# Explore

In [3]:
train_data.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [4]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Pre-processing

In [6]:
encoded_train = train_data.copy()
encoded_test_X = test_X.copy()
encoded_test_y = test_y.copy()

In [7]:
def delete_col(df,col):
    for i in col:
        df.drop(i,axis = 1,inplace = True)

In [8]:
col = ['PassengerId','Name','Ticket','Cabin']
delete_col(encoded_train,col)
delete_col(encoded_test_X,col)

In [9]:
def fill_int(df,col):
    for i in col:
        q1 = df[i].quantile(0.25)
        q3 = df[i].quantile(0.75)
        iqr = q3 - q1
        lower_lim = q1 - 1.5*iqr
        upper_lim = q3 + 1.5*iqr
        without_outlier = df[i].apply(lambda x : None if x > upper_lim or x < lower_lim else x)
        without_outlier.fillna(without_outlier.mean(),inplace = True)
        df[i] = without_outlier
def fill_obj(df,col):
    for i in col:
        df[i].fillna(df[i].mode()[0],inplace = True)

In [10]:
def int_col(df):
    col = []
    for i in df.columns:
        if df[i].dtype != 'O':
            col.append(i)
    return col
def obj_col(df):
    col = []
    for i in df.columns:
        if df[i].dtype == 'O':
            col.append(i)
    return col

In [11]:
col1 = int_col(encoded_train)
col2 = int_col(encoded_test_X)
fill_int(encoded_train,col1)
fill_int(encoded_test_X,col2)

In [12]:
col1 = obj_col(encoded_train)
col2 = obj_col(encoded_test_X)
fill_obj(encoded_train,col1)
fill_obj(encoded_test_X,col2)

C:\Users\Asaad\AppData\Local\Temp\ipykernel_9784\4093639794.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].fillna(df[i].mode()[0],inplace = True)


In [13]:
col = ['PassengerId']
delete_col(encoded_test_y,col)

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
def obj_to_int(df,col):
    label = LabelEncoder()
    for i in col:
        df[i] = label.fit_transform(df[i])

In [16]:
col1 = obj_col(encoded_train)
obj_to_int(encoded_train,col1)
col2 = obj_col(encoded_test_X)
obj_to_int(encoded_test_X,col2)

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
minmax = MinMaxScaler()

In [19]:
col = ['Pclass','Age','SibSp','Parch','Fare']
for i in col:
    encoded_train[col] = minmax.fit_transform(encoded_train[col])

In [20]:
col = ['Pclass','Age','SibSp','Parch','Fare']
for i in col:
    encoded_test_X[col] = minmax.fit_transform(encoded_test_X[col])

# Training 

In [21]:
train_X = encoded_train.drop('Survived',axis=1)
train_y = encoded_train['Survived']

# Model trianig

In [22]:
from sklearn.svm import SVC

In [23]:
svc_model = SVC()
svc_model.fit(train_X,train_y)

SVC()

In [24]:
pred = svc_model.predict(encoded_test_X)

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
acc = accuracy_score(encoded_test_y,pred)
acc

0.9019138755980861

In [27]:
test_y['Survived'] = pred

In [28]:
test_y.to_csv('Titanic_submission.csv',index = False)

In [29]:
# Add these to your existing training code
import joblib

# Save Label Encoders
label_encoders = {}
for col in obj_col(encoded_train):
    le = LabelEncoder()
    encoded_train[col] = le.fit_transform(encoded_train[col])
    label_encoders[col] = le
joblib.dump(label_encoders, 'label_encoders.pkl')

# Save Scaler
scaler = MinMaxScaler()
cols_to_scale = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
encoded_train[cols_to_scale] = scaler.fit_transform(encoded_train[cols_to_scale])
joblib.dump(scaler, 'scaler.pkl')

# Save Model
joblib.dump(svc_model, 'titanic_model.pkl')

['titanic_model.pkl']